In [10]:
import requests
import json

address = 'https://edr-api-c.mdl.nws.noaa.gov/Climate-EDR/'

In [12]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [6]:
response = requests.get(address + 'collections')

In [20]:
for i in response.json()['collections']:
    print(i['title'])

nasa-gddp
nclimgrid-monthly
livneh-monthly
loca2-monthly-historical
loca2-monthly-future


In [38]:
collection = 'loca2-monthly-future'

In [41]:
c = response.json()['collections'][4]  
for param in c['parameter_names']:
    p=c['parameter_names'][param]
    print(p['description'])

pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
pr
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmax
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin
tasmin


In [49]:
c

{'links': [{'href': 'https://www.weather.gov/disclaimer',
   'hreflang': 'en',
   'rel': 'terms-of-use',
   'type': 'text/html'},
  {'href': 'https://loca.ucsd.edu/loca-bibliography/',
   'hreflang': 'en',
   'rel': 'citation',
   'type': 'text/html'}],
 'id': 'loca2-monthly-future',
 'title': 'loca2-monthly-future',
 'description': 'loca2-monthly-future',
 'keywords': ['pr_ipsl_cm6a_lr_ssp245_r1i1p1f1',
  'pr_ipsl_cm6a_lr_ssp245_r2i1p1f1',
  'pr_ipsl_cm6a_lr_ssp245_r3i1p1f1',
  'pr_ipsl_cm6a_lr_ssp245_r4i1p1f1',
  'pr_ipsl_cm6a_lr_ssp245_r5i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r10i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r1i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r2i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r3i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r4i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r5i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r6i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r7i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r8i1p1f1',
  'pr_ipsl_cm6a_lr_ssp370_r9i1p1f1',
  'pr_ipsl_cm6a_lr_ssp585_r1i1p1f1',
  'pr_ipsl_cm6a_lr_ssp585_r2i1p1f1',

In [46]:
response = requests.get(address + 'collections/loca2-monthly-future/items')

In [47]:
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)